In [5]:
library(tidyverse)
library(rnoaa)

# pull stations that are located in similar latitude to Kyoto (35.01)
station_df <- read.csv("../A_outputs/A31_stations.csv") %>%
    filter(str_sub(id, start = 1, end = 2) == "JA") %>%
    filter(34.7 <latitude & latitude < 35.3) %>%
    filter(2021 < last_year)
station_cities <- station_df$name
# dim(station_df)
# station_df

# pull cherry_sub.csv
cherry_sub <- read.csv("../A_outputs/A11_cherry_sub.csv") %>%
    filter(country == "Japan") %>%
    filter(toupper(city) %in% station_cities)

cities <- unique(cherry_sub$city)
station_ids <- station_df[station_df$name %in% toupper(cherry_sub$city), "id"]
station_ids # 9 station ids

id_city <- station_df[station_df$name %in% toupper(cherry_sub$city), c("id", "name", "latitude", "longitude", "elevation")]
write.csv(id_city, "../A_outputs/A31_japan_station_city.csv")

Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“EOF within quoted string”


[1] "JA000047636" "JA000047651" "JA000047654" "JA000047656" "JA000047672"
[6] "JA000047675" "JA000047755" "JA000047759" "JA000047761"

In [7]:
head(cherry_sub)

,location,lat,long,alt,year,bloom_date,bloom_doy,country,city
,<chr>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<int>,<chr>,<chr>
1,Japan/Nagoya,35.16778,136.9656,51.1,1953,1953-04-12,102,Japan,Nagoya
2,Japan/Nagoya,35.16833,136.9647,51.1,1953,1953-04-12,102,Japan,Nagoya
3,Japan/Nagoya,35.16778,136.9656,51.1,1954,1954-04-04,94,Japan,Nagoya
4,Japan/Nagoya,35.16833,136.9647,51.1,1954,1954-04-04,94,Japan,Nagoya
5,Japan/Nagoya,35.16778,136.9656,51.1,1955,1955-04-03,93,Japan,Nagoya
6,Japan/Nagoya,35.16833,136.9647,51.1,1955,1955-04-03,93,Japan,Nagoya


In [6]:
id_city

,id,name,latitude,longitude,elevation
,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,JA000047636,NAGOYA,35.167,136.967,56
3,JA000047651,TSU,34.733,136.517,18
4,JA000047654,HAMAMATSU,34.717,137.717,33
5,JA000047656,SHIZUOKA,34.983,138.400,15
8,JA000047672,TATEYAMA,34.983,139.867,7
10,JA000047675,OSHIMA,34.750,139.383,79
12,JA000047755,HAMADA,34.900,132.067,20
14,JA000047759,KYOTO,35.017,135.733,46
15,JA000047761,HIKONE,35.283,136.250,89


In [37]:
# Obtain Historic daily temperature
library(tidyverse)
library(rnoaa)

get_temperature <- function (stationid) {
    dat <- ghcnd_search(stationid = stationid, var = c("TMAX", "TMIN", "PRCP"), 
               date_min = "1950-01-01", date_max = "2023-01-31") %>%
               purrr::reduce(left_join, by = "date") %>%
               select(id.x, date, tmax, tmin, prcp) %>%
               rename_with(~ "id", id.x) %>%
               mutate(tmax = tmax/10) %>%      # in C
               mutate(tmin = tmin/10) %>%      # in C
               mutate(prcp = prcp/10) %>%      # in mm
               mutate(year = format(date, "%Y")) %>%
               mutate(month = as.integer(strftime(date, '%m'))) %>%
               mutate(day = as.integer(strftime(date, '%d')))
    
    return(dat)
}


city_temp_list <- list()

for (c in seq_len(length(cities))) {

    temp_df <- get_temperature(station_ids[c])

    # Impute missing data
    library(mice)

    # check missing data
    # md.pattern(temp_df)

    tempData <- mice(temp_df, m = 5, method = "pmm", seed = 1)
    # summary(tempData)

    imputed_temp <- complete(tempData, 5)
    # imputed_file_name <- paste0("/home/joosungm/projects/def-lelliott/joosungm/projects/peak-bloom-prediction/A_outputs/A21_", cities[c], "_temperature.csv")
    # print(imputed_file_name)
    # write.csv(imputed_temp, imputed_file_name)

    city_temp_list[[c]] <- imputed_temp

}

head(city_temp_list[[9]])

using cached file: /home/jovyan/.cache//R/noaa_ghcnd/JA000047636.dly

date created (size, mb): 2023-02-09 03:08:07 (1.721)

file min/max dates: 1951-01-01 / 2022-10-31




 iter imp variable
  1   1  tmax  tmin  prcp
  1   2  tmax  tmin  prcp
  1   3  tmax  tmin  prcp
  1   4  tmax  tmin  prcp
  1   5  tmax  tmin  prcp
  2   1  tmax  tmin  prcp
  2   2  tmax  tmin  prcp
  2   3  tmax  tmin  prcp
  2   4  tmax  tmin  prcp
  2   5  tmax  tmin  prcp
  3   1  tmax  tmin  prcp
  3   2  tmax  tmin  prcp
  3   3  tmax  tmin  prcp
  3   4  tmax  tmin  prcp
  3   5  tmax  tmin  prcp
  4   1  tmax  tmin  prcp
  4   2  tmax  tmin  prcp
  4   3  tmax  tmin  prcp
  4   4  tmax  tmin  prcp
  4   5  tmax  tmin  prcp
  5   1  tmax  tmin  prcp
  5   2  tmax  tmin  prcp
  5   3  tmax  tmin  prcp
  5   4  tmax  tmin  prcp
  5   5  tmax  tmin  prcp


Warning message:
“Number of logged events: 2”
using cached file: /home/jovyan/.cache//R/noaa_ghcnd/JA000047651.dly

date created (size, mb): 2023-02-09 03:09:19 (1.536)

file min/max dates: 1951-01-01 / 2022-10-31




 iter imp variable
  1   1  tmax  tmin  prcp
  1   2  tmax  tmin  prcp
  1   3  tmax  tmin  prcp
  1   4  tmax  tmin  prcp
  1   5  tmax  tmin  prcp
  2   1  tmax  tmin  prcp
  2   2  tmax  tmin  prcp
  2   3  tmax  tmin  prcp
  2   4  tmax  tmin  prcp
  2   5  tmax  tmin  prcp
  3   1  tmax  tmin  prcp
  3   2  tmax  tmin  prcp
  3   3  tmax  tmin  prcp
  3   4  tmax  tmin  prcp
  3   5  tmax  tmin  prcp
  4   1  tmax  tmin  prcp
  4   2  tmax  tmin  prcp
  4   3  tmax  tmin  prcp
  4   4  tmax  tmin  prcp
  4   5  tmax  tmin  prcp
  5   1  tmax  tmin  prcp
  5   2  tmax  tmin  prcp
  5   3  tmax  tmin  prcp
  5   4  tmax  tmin  prcp
  5   5  tmax  tmin  prcp


Warning message:
“Number of logged events: 2”
using cached file: /home/jovyan/.cache//R/noaa_ghcnd/JA000047654.dly

date created (size, mb): 2023-02-09 03:09:26 (1.574)

file min/max dates: 1951-01-01 / 2023-02-28



...


 iter imp variable
  1   1  tmax  tmin  prcp
  1   2  tmax  tmin  prcp
  1   3  tmax  tmin  prcp
  1   4  tmax  tmin  prcp
  1   5  tmax  tmin  prcp
  2   1  tmax  tmin  prcp
  2   2  tmax  tmin  prcp
  2   3  tmax  tmin  prcp
  2   4  tmax  tmin  prcp
  2   5  tmax  tmin  prcp
  3   1  tmax  tmin  prcp
  3   2  tmax  tmin  prcp
  3   3  tmax  tmin  prcp
  3   4  tmax  tmin  prcp
  3   5  tmax  tmin  prcp
  4   1  tmax  tmin  prcp
  4   2  tmax  tmin  prcp
  4   3  tmax  tmin  prcp
  4   4  tmax  tmin  prcp
  4   5  tmax  tmin  prcp
  5   1  tmax  tmin  prcp
  5   2  tmax  tmin  prcp
  5   3  tmax  tmin  prcp
  5   4  tmax  tmin  prcp
  5   5  tmax  tmin  prcp


Warning message:
“Number of logged events: 2”
using cached file: /home/jovyan/.cache//R/noaa_ghcnd/JA000047761.dly

date created (size, mb): 2023-02-09 03:10:07 (1.536)

file min/max dates: 1951-01-01 / 2023-02-28




 iter imp variable
  1   1  tmax  tmin  prcp
  1   2  tmax  tmin  prcp
  1   3  tmax  tmin  prcp
  1   4  tmax  tmin  prcp
  1   5  tmax  tmin  prcp
  2   1  tmax  tmin  prcp
  2   2  tmax  tmin  prcp
  2   3  tmax  tmin  prcp
  2   4  tmax  tmin  prcp
  2   5  tmax  tmin  prcp
  3   1  tmax  tmin  prcp
  3   2  tmax  tmin  prcp
  3   3  tmax  tmin  prcp
  3   4  tmax  tmin  prcp
  3   5  tmax  tmin  prcp
  4   1  tmax  tmin  prcp
  4   2  tmax  tmin  prcp
  4   3  tmax  tmin  prcp
  4   4  tmax  tmin  prcp
  4   5  tmax  tmin  prcp
  5   1  tmax  tmin  prcp
  5   2  tmax  tmin  prcp
  5   3  tmax  tmin  prcp
  5   4  tmax  tmin  prcp
  5   5  tmax  tmin  prcp


Warning message:
“Number of logged events: 2”


,id,date,tmax,tmin,prcp,year,month,day
,<chr>,<date>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<int>
1,JA000047761,1951-01-01,4.5,1.4,10.4,1951,1,1
2,JA000047761,1951-01-02,4.1,0.1,1.8,1951,1,2
3,JA000047761,1951-01-03,6.4,-1.8,0.0,1951,1,3
4,JA000047761,1951-01-04,5.0,3.3,0.9,1951,1,4
5,JA000047761,1951-01-05,5.7,3.4,1.2,1951,1,5
6,JA000047761,1951-01-06,6.3,1.1,0.0,1951,1,6


In [35]:
tail(city_temp_list[[5]])

,id,date,tmax,tmin,prcp,year,month,day
,<chr>,<date>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<int>
19719,JA000047672,2023-01-26,9.1,-5.0,0.0,2023,1,26
19720,JA000047672,2023-01-27,10.7,-2.5,7.1,2023,1,27
19721,JA000047672,2023-01-28,8.5,2.5,0.0,2023,1,28
19722,JA000047672,2023-01-29,8.3,-2.6,0.0,2023,1,29
19723,JA000047672,2023-01-30,11.8,0.8,0.0,2023,1,30
19724,JA000047672,2023-01-31,9.3,-2.9,0.0,2023,1,31


In [39]:
temperature_df <- city_temp_list %>% bind_rows()
dim(temperature_df)
data.table::fwrite(temperature_df, file = "../A_outputs/A31_japan_temperature.csv", row.names = FALSE, quote = FALSE)

[1] 223177      8